In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import nltk
from nltk.stem.cistem import Cistem
import string
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *
import matplotlib.pyplot as plt
import seaborn
from textblob_de import TextBlobDE as TextBlob
import warnings
import h2o
from h2o.automl import H2OAutoML
import random 
import os
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
%matplotlib inline

In [2]:
# Raw data
train_data = pd.read_csv("/home/mackenzie/Downloads/GermanTrainingData.txt", sep='\t', names=['tweet', 'coarse', 'labels'])
test_data = pd.read_csv("/home/mackenzie/Downloads/GermanTestingData.txt", sep='\t', names=['tweet', 'coarse', 'labels'])
df = pd.concat([train_data, test_data], ignore_index=True)
del train_data
del test_data

In [3]:
tweets=df.tweet

In [4]:
stopwords=stopwords = nltk.corpus.stopwords.words("german")

other_exclusions = ["lbr", "|lbr|", "»"]
stopwords.extend(other_exclusions)

stemmer = Cistem()

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-ßA-ß]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-ßA-ß]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-ßA-ß.,!?]+", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords,
    use_idf=True,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=3000,
    min_df=5,
    max_df=0.75
    )

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
print(vocab) # todo: add 
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['all', 'ber', 'dami', 'dasselb', 'demselb', 'denselb', 'derselb', 'dess', 'desselb', 'dieselb', 'dor', 'etwa', 'eur', 'f', 'geg', 'hatt', 'hrend', 'jed', 'jen', 'jetz', 'k', 'mach', 'manch', 'nich', 'nne', 'nnt', 'ohn', 'r', 'rde', 'selb', 'solch', 'son', 'sond', 'w', 'welch', 'werd', 'wes', 'woll', 'zwisch'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'^ml': 0, '`ne': 1, 'a': 2, 'a h': 3, 'ab': 4, 'abartig': 5, 'abend': 6, 'abgefuck': 7, 'abgefuck spd': 8, 'abgeleh': 9, 'abgeord': 10, 'abgeschaff': 11, 'abgeschob': 12, 'abgeschob werd': 13, 'abgeseh': 14, 'abgew': 15, 'abgew hlt': 16, 'abh': 17, 'ableg': 18, 'abschaff': 19, 'abschaffung': 20, 'abschaum': 21, 'abschieb': 22, 'abschiebung': 23, 'absehbar': 24, 'absich': 25, 'absolu': 26, 'abstimmung': 27, 'absurd': 28, 'abu': 29, 'abw': 30, 'abwar': 31, 'abzug': 32, 'accou': 33, 'ach': 34, 'ach ja': 35, 'ach scheiss': 36, 'achtung': 37, 'adolf': 38, 'adv': 39, 'afd': 40, 'afd nich': 41, 'afd w': 42, 'afd w hle': 43, 'afd w hler': 44, 'aff': 45, 'afghanista': 46, 'afri': 47, 'afrika': 48, 'agenda': 49, 'aha': 50, 'ahnung': 51, 'aksynod': 52, 'akt': 53, 'aktio': 54, 'aktuell': 55, 'akzeptier': 56, 'al': 57, 'alkohol': 58, 'all': 59, 'all ber': 60, 'all f': 61, 'all gut': 62, 'all m': 63, 'all mensch': 64, 'all mittel': 65, 'all nich': 66, 'all schon': 67, 'all w': 68, 'allei': 69, 'all

In [ ]:
#Now get other features
sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-ßA-ß]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = TextBlob("tweet").sentiment
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words, lang='de_DE')
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet)
    features = [FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment[0],twitter_objs[2], twitter_objs[1], twitter_objs[0]]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

# Changing string labels to numeric labels
def string_to_numeric(x):
    if x == 'OTHER' or x == 'PROFANITY':
        return 0
    if x == 'INSULT':
        return 1
    if x == 'ABUSE':
        return 2

In [ ]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "sentiment", "num_hashtags", "num_mentions", "num_urls"]

In [ ]:
feats = get_feature_array(tweets)

In [ ]:
#Now join them all up
M = np.concatenate([tfidf,feats],axis=1)

In [ ]:
M.shape

(8407, 3013)

In [ ]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

feature_names = variables+other_features_names

In [ ]:
X = pd.DataFrame(M)
y = df['labels'].apply(string_to_numeric)
X.columns = feature_names

In [ ]:
# Univariate feature selection from sklearn
bestfeatures = SelectKBest(score_func=f_classif, k=1000)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

# concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print('Univariate Selection features found, use getUnivariateData() to get the features')
# Extract the top n features
uni_selected_feat = featureScores.nlargest(1000,'Score')
print(uni_selected_feat) # print out the top n features selected

# Saving the top n features to a data frame
#print(a.iloc[0].name) # how to get the column # for the ith feature
#print(a.iloc[0][0]) # how to get the header column
top_univariate_features = pd.DataFrame()
for i in range(0, 1000):
    curr_column_vals = X.iloc[:, uni_selected_feat.iloc[i].name]
    curr_column_name = uni_selected_feat.iloc[i][0]
    top_univariate_features[curr_column_name] = curr_column_vals

X = pd.DataFrame(top_univariate_features)
X.columns = top_univariate_features.columns
print(X.columns)
feature_names = top_univariate_features.columns
featuresSelected_pd = pd.DataFrame(feature_names)

/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [3009 3012] are constant.
  UserWarning)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Univariate Selection features found, use getUnivariateData() to get the features
                   Specs      Score
614                 dumm  64.461365
2525          strunzdumm  55.880652
1704                mosl  55.345552
587               dophil  54.787880
2275           schmarotz  53.819002
1960                pack  52.935005
1956            p dophil  52.840050
1659              merkel  52.572181
1263             invasor  49.851795
1278               islam  48.541379
164                asyla  47.059928
1717             murksel  45.142919
1898              npress  45.073805
1488            l npress  43.138647
2703              vasall  41.965389
2254             scheiss  41.374236
2300      schulabschluss  38.508092
2798           volksverr  35.600844
831               fettig  34.680448
832          fettig haar  34.563703
2276         schmarotz p  34.354193
2277  schmarotz p dophil  34.354193
2303              schulz  33.965406
21              abschaum  33.503491
1955               

In [ ]:
print(len(X))
print(len(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
y_train = y_train.to_frame(name='labels')
y_test = y_test.to_frame(name='labels')

In [ ]:
# Undersampling from imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
X_train = pd.DataFrame(X_res)
X_train.columns = feature_names
y_train = pd.DataFrame(y_res)
y_train.columns = ['labels']

In [ ]:
h2o.init()

In [ ]:
# BUG: when we convert to h2o dataframe an extra row is added
X_train = h2o.H2OFrame(X_train)
y_train = h2o.H2OFrame(y_train)
X_test = h2o.H2OFrame(X_test)
y_test = h2o.H2OFrame(y_test) 

In [ ]:
X_train = X_train.na_omit() # pandas-->h2o results in an error where a nan row is added to X data, so this deals with that
X_test = X_test.na_omit()

In [ ]:
# Double check X and y have same number of rows
print(len(y_train))
print(len(X_train))
print(len(y_test))
print(len(X_test))

In [ ]:
X_train.shape

In [ ]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)

In [ ]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_runtime_secs=1800) #sort_metric=auc, max_runtime_secs=10800, class_sampling_factors = sample_factors, max_models=?, balance_classes=True
aml.train(x=x, y=y, training_frame=train)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
lb_pd = lb.as_data_frame()

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
# predictions!
preds = aml.predict(test)
print(preds)
var = preds["predict"].cbind(test[y])
print(var)

In [ ]:
# Create a new folder to hold statistics for a run
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# create a random folder number to reference the directory
rand = random.randint(0, 1000)*random.randint(0,10)
directory = '/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/TestResults_replicated/Test_' + str(rand)
print('The results for this run will be stored in ' + directory)
createFolder(directory)

In [ ]:
# metrics and results!
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print("Confusion Matrix: ")
print(metrics.confusion_matrix(y_test, y_pred))
print("Accuracy Score: ")
print(metrics.accuracy_score(y_test, y_pred))
print("F1 Score: ")
print(metrics.f1_score(y_test, y_pred, average="weighted"))
print("Recall: ")
confusion_matrix = metrics.confusion_matrix(y_test,y_pred)
matrix_proportions = np.zeros((3,3))
for i in range(0,3):
    matrix_proportions[i,:] = confusion_matrix[i,:]/float(confusion_matrix[i,:].sum())
names=['Other','Insult','Abuse']
confusion_df = pd.DataFrame(matrix_proportions, index=names,columns=names)
plt.figure(figsize=(5,5))
seaborn.heatmap(confusion_df,annot=True,annot_kws={"size": 12},cmap='gist_gray_r',cbar=False, square=True,fmt='.2f')
plt.ylabel(r'True categories',fontsize=14)
plt.xlabel(r'Predicted categories',fontsize=14)
plt.tick_params(labelsize=12)
plt.savefig(directory + '/recall.png')

In [ ]:
# Saves the classification report info
print(metrics.classification_report(y_test, y_pred))
f= open(directory+'/classification_report.txt',"w+")
f.write(metrics.classification_report(y_test, y_pred))
f.write(str(metrics.confusion_matrix(y_test,y_pred)))
f.write('\n')
f.write('accuracy: '+ str(metrics.accuracy_score(y_test, y_pred)))
f.write('\n')
f.write('f1-score' + str(metrics.f1_score(y_test, y_pred, average="weighted")))
f.write('\n')
f.close()

In [ ]:
file = lb_pd.to_csv(path_or_buf=directory+'/modelinfo.csv')
featuresSelected = featuresSelected_pd.to_csv(path_or_buf=directory+'/feature_selection.csv')